In [6]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from buck.analysis.basics import ingest_images

# Your existing ingestion
fpath = "..\\images\\squared\\*_NDA.png"
images, ages = ingest_images(fpath)

In [7]:
from buck.analysis.basics import split_data

Xtr_og, ytr_og, Xval, yval, Xte, yte_onehot, ages, l_map = split_data(images, ages)

(143, 288, 288, 1) (36, 288, 288, 1) (45, 288, 288, 1)


In [14]:
from buck.analysis.basics import homogenize_data

augment_multiplier = 40
X_train_pca, y_train_flat, X_test_pca, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, augment_multiplier)

  Class 0 (Age 1.5): 20 → 1360 samples
  Class 1 (Age 2.5): 25 → 1360 samples
  Class 2 (Age 3.5): 30 → 1360 samples
  Class 3 (Age 4.5): 34 → 1360 samples
  Class 4 (Age 5.5): 34 → 1360 samples


In [15]:
from buck.classifiers.random_forest import (
    _optimize_rs, _optimize_nest, _optimize_max_d, _optimize_crit, _optimize_cw, _optimize_mss, _optimize_msl, _optimize_mwfl, _optimize_mf, _optimize_mln, _optimize_mid
)

# Optimize Augmentation factor (RandFor)

for aug_mul in np.arange(1,65,1):
    X_train_pca, y_train_flat, X_test_pca, y_true, label_mapping, num_classes = homogenize_data(Xtr_og, ytr_og, Xte,yte_onehot, l_map, aug_mul)

    # Shorten parameters
    Xtr_pca = X_train_pca
    ytr_flat = y_train_flat
    Xte_pca = X_test_pca

    opts = {"n_estimators": 100, "criterion": "gini", "max_depth": None, "min_samples_split": 2, "min_samples_leaf": 1, "min_weight_fraction_leaf": 0.0,
            "max_features": "sqrt", "max_leaf_nodes": None, "min_impurity_decrease": 0.0, "bootstrap": True, "oob_score": False, "n_jobs": -1,
            "random_state": 42, "verbose": 0, "warm_start": False, "class_weight": None, "ccp_alpha": 0.0, "max_samples": None, "monotonic_cst": None}
    
    # Optimize hyperparameters
    ma_vec = []
    f1_vec = []
    opts, _, _ = _optimize_rs(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_nest(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_max_d(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_crit(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)  # type: ignore
    opts, _, _ = _optimize_cw(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mss(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_msl(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mwfl(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mf(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mln(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, ma, f1 = _optimize_mid(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    ma_vec.append(ma)
    f1_vec.append(f1)

    print(aug_mul,ma,f1)

  Class 0 (Age 1.5): 20 → 34 samples
  Class 1 (Age 2.5): 25 → 34 samples
  Class 2 (Age 3.5): 30 → 34 samples
  Class 3 (Age 4.5): 34 → 34 samples
  Class 4 (Age 5.5): 34 → 34 samples
1 0.5333333333333333 0.5285075433682245
  Class 0 (Age 1.5): 20 → 68 samples
  Class 1 (Age 2.5): 25 → 68 samples
  Class 2 (Age 3.5): 30 → 68 samples
  Class 3 (Age 4.5): 34 → 68 samples
  Class 4 (Age 5.5): 34 → 68 samples
2 0.6 0.6029614634877792
  Class 0 (Age 1.5): 20 → 102 samples
  Class 1 (Age 2.5): 25 → 102 samples
  Class 2 (Age 3.5): 30 → 102 samples
  Class 3 (Age 4.5): 34 → 102 samples
  Class 4 (Age 5.5): 34 → 102 samples
3 0.5777777777777777 0.5756956493798598
  Class 0 (Age 1.5): 20 → 136 samples
  Class 1 (Age 2.5): 25 → 136 samples
  Class 2 (Age 3.5): 30 → 136 samples
  Class 3 (Age 4.5): 34 → 136 samples
  Class 4 (Age 5.5): 34 → 136 samples
4 0.5555555555555556 0.5405539993775288
  Class 0 (Age 1.5): 20 → 170 samples
  Class 1 (Age 2.5): 25 → 170 samples
  Class 2 (Age 3.5): 30 → 170

MemoryError: Unable to allocate 1.63 GiB for an array with shape (5270, 82944) and data type float32

In [ ]:
from buck.classifiers.autotune import optimize_all

results = optimize_all(X_train_pca, y_train_flat, X_test_pca, y_true, cycles = 1)
print(results)

Optimizing
...adaboost


## Homogenize data across classes

## Optimize all classifier models

In [ ]:
# AdaBoost
'''
from buck.classifiers.ada_boost import (_optimize_rs, _optimize_nest, _optimize_lr)

# Shorten parameters
Xtr_pca = X_train_pca
ytr_flat = y_train_flat
Xte_pca = X_test_pca

# Define optimals
opts = {
    "random_state": None,
    "estimator": None,
    "n_estimators": 50,
    "learning_rate": 1.0,
}

# Adaboost
opts, ma, ab_rs = _optimize_rs(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
opts, ma, ab_ne = _optimize_nest(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
opts, ma, ab_lr = _optimize_lr(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
print(ma)
'''

In [ ]:
# Random Forest
from buck.classifiers.random_forest import (
    _optimize_rs, _optimize_nest, _optimize_max_d, _optimize_crit, _optimize_cw, _optimize_mss, _optimize_msl, _optimize_mwfl, _optimize_mf, _optimize_mln, _optimize_mid
)

# Shorten parameters
Xtr_pca = X_train_pca
ytr_flat = y_train_flat
Xte_pca = X_test_pca

opts = {
    "n_estimators": 100,
    "criterion": "gini",
    "max_depth": None,
    "min_samples_split": 2,
    "min_samples_leaf": 1,
    "min_weight_fraction_leaf": 0.0,
    "max_features": "sqrt",
    "max_leaf_nodes": None,
    "min_impurity_decrease": 0.0,
    "bootstrap": True,
    "oob_score": False,
    "n_jobs": -1,
    "random_state": 42,
    "verbose": 0,
    "warm_start": False,
    "class_weight": None,
    "ccp_alpha": 0.0,
    "max_samples": None,
    "monotonic_cst": None,
}

# Optimize hyperparameters
ma_vec = []
f1_vec = []
for c in np.arange(10):
    opts, _, _ = _optimize_rs(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_nest(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_max_d(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_crit(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)  # type: ignore
    opts, _, _ = _optimize_cw(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mss(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_msl(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mwfl(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mf(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, _, _ = _optimize_mln(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    opts, ma, f1 = _optimize_mid(Xtr_pca, ytr_flat, Xte_pca, y_true, opts)
    ma_vec.append(ma)
    f1_vec.append(f1)
    print(ma, f1)

#Best: 0.7894736842105263
#{'n_estimators': np.int64(127),
# 'criterion': 'gini',
# 'max_depth': None,
# 'min_samples_split': np.int64(2),
# 'min_samples_leaf': np.int64(1),
# 'min_weight_fraction_leaf': np.float64(0.0),
# 'max_features': 'log2',
# 'max_leaf_nodes': None,
# 'min_impurity_decrease': np.float64(0.0),
# 'bootstrap': True,
# 'oob_score': False,
# 'n_jobs': -1,
# 'random_state': np.int64(405),
# 'verbose': 0,
# 'warm_start': False,
# 'class_weight': None,
# 'ccp_alpha': 0.0,
# 'max_samples': None,
# 'monotonic_cst': None}

In [ ]:
# Neural Net
'''
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

# Shorten parameters
Xtr_pca = X_train_pca
ytr_flat = y_train_flat
Xte_pca = X_test_pca

# Define optimals
opts = {
    "hidden_layer_sizes": (100,),
    "activation": "relu",
    "solver": "adam",
    "alpha": 0.0001,
    "batch_size": "auto",
    "learning_rate": "constant",
    "learning_rate_init": 0.001,
    "power_t": 0.5,
    "max_iter": 20000,
    "shuffle": True,
    "random_state": None,
    "tol": 0.01,
    "verbose": False,
    "warm_start": False,
    "momentum": 0.9,
    "nesterovs_momentum": True,
    "early_stopping": False,
    "validation_fraction": 0.1,
    "beta_1": 0.9,
    "beta_2": 0.999,
    "epsilon": 1e-08,
    "n_iter_no_change": 10,
    "max_fun": 15000,
}

# Initialize variables
ac_vec = []
f1_vec = []
max_acc = -np.inf
max_idx = -1
variable_array = np.arange(150)
#best_val = variable_array[0]
for i in np.arange(len(variable_array)):
    v = variable_array[i]
    # Define classifiers to test
    classifier = MLPClassifier(
        random_state=v,
        hidden_layer_sizes=opts["hidden_layer_sizes"],
        activation=opts["activation"],
        solver=opts["solver"],
        alpha=opts["alpha"],
        batch_size=opts["batch_size"],
        learning_rate=opts["learning_rate"],
        learning_rate_init=opts["learning_rate_init"],
        power_t=opts["power_t"],
        max_iter=opts["max_iter"],
        shuffle=opts["shuffle"],
        tol=opts["tol"],
        verbose=opts["verbose"],
        warm_start=opts["warm_start"],
        momentum=opts["momentum"],
        nesterovs_momentum=opts["nesterovs_momentum"],
        early_stopping=opts["early_stopping"],
        validation_fraction=opts["validation_fraction"],
        beta_1=opts["beta_1"],
        beta_2=opts["beta_2"],
        epsilon=opts["epsilon"],
        n_iter_no_change=opts["n_iter_no_change"],
        max_fun=opts["max_fun"],
    )
    # Train the classifier
    classifier.fit(X_train_pca, y_train_flat)
    # Make predictions
    y_pred = classifier.predict(X_test_pca)
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    ac_vec.append(accuracy)
    f1 = f1_score(y_true, y_pred, average="weighted", zero_division=0)
    f1_vec.append(f1)
    
    # Return index
    if accuracy >= max_acc:
        max_acc = accuracy
        print(max_acc)
        best_val = v
    
    # Store best value
    opts["random_state"] = best_val
'''

In [ ]:
#from buck.classifiers.compare_models import compare_models
#
## Shorten parameters
#Xtr_pca = X_train_pca
#ytr_flat = y_train_flat
#Xte_pca = X_test_pca
#
#compare_models(Xtr_pca, ytr_flat, Xte_pca, y_true, num_classes, label_mapping)